# Sales strategy analysis for "Pens and Printers"

We are tasked with analyzing the effectiveness of three sales strategies used by "Pens and Printers" for their new stationery product line. By evaluating customer reach and revenue generation, our goal is to recommend the optimal sales approach to enhance future product launches.

## 1. Import packages

In [1]:
# import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re
from scipy import stats
from datetime import datetime

# render figures in notebook
%matplotlib inline

## 2. Load dataset and data preview

In [2]:
# load dataset
data = pd.read_csv('product_sales.csv')

In [3]:
# preview first rows
data.head()

,week,sales_method,customer_id,nb_sold,revenue,years_as_customer,nb_site_visits,state
0,2,Email,2e72d641-95ac-497b-bbf8-4861764a7097,10,NaN,0,24,Arizona
1,6,Email + Call,3998a98d-70f5-44f7-942e-789bb8ad2fe7,15,225.47,1,28,Kansas
2,5,Call,d1de9884-8059-4065-b10f-86eef57e4a44,11,52.55,6,26,Wisconsin
3,4,Email,78aa75a4-ffeb-4817-b1d0-2f030783c5d7,11,NaN,3,25,Indiana
4,3,Email,10e6d446-10a5-42e5-8210-1b5438f70922,9,90.49,0,28,Illinois


## 3. Initial basic EDA

In [4]:
# data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   week               15000 non-null  int64  
 1   sales_method       15000 non-null  object 
 2   customer_id        15000 non-null  object 
 3   nb_sold            15000 non-null  int64  
 4   revenue            13926 non-null  float64
 5   years_as_customer  15000 non-null  int64  
 6   nb_site_visits     15000 non-null  int64  
 7   state              15000 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 937.6+ KB


In [5]:
# basic statistics (numeric variables)
data.describe()

,week,nb_sold,revenue,years_as_customer,nb_site_visits
count,15000.000000,15000.000000,13926.000000,15000.000000,15000.000000
mean,3.098267,10.084667,93.934943,4.965933,24.990867
std,1.656420,1.812213,47.435312,5.044952,3.500914
min,1.000000,7.000000,32.540000,0.000000,12.000000
25%,2.000000,9.000000,52.470000,1.000000,23.000000
50%,3.000000,10.000000,89.500000,3.000000,25.000000
75%,5.000000,11.000000,107.327500,7.000000,27.000000
max,6.000000,16.000000,238.320000,63.000000,41.000000


From the initial basic EDA we know that:

- the dataset has 15'000 entries
- the dataset has 8 columns: 'week', 'sales_method', 'customer_id', 'nb_sold', 'revenue', 'years_as_customer', 'nb_site_visits' and 'state'
- the dataset contains numeric and categorical variables
- there are no negative values within the numeric variables
- 'revenue' column has 1074 missing values

## 4. Cleaning steps and data validation

We are now ready to perform data wrangling and data validation according to the table information provided.

### 4.1 Dataset overview

#### 4.1.1 Data types

In [6]:
# actual dtypes
actual_dtypes = data.dtypes

# expected dtypes
expected_dtypes = {
    'week': 'int64',
    'sales_method': 'object', # 'character' type corresponds to 'object' dtype in pandas
    'customer_id': 'object',
    'nb_sold': 'int64',
    'revenue': 'float64', # requires 2 decimals which makes 'numeric' a 'float' dtype
    'years_as_customer': 'int64',
    'nb_site_visits': 'int64',
    'state': 'object'
}

# check actual vs expected dtypes
dtype_check = actual_dtypes == pd.Series(expected_dtypes)

In [7]:
# dtype check dataframe
dtype_df = pd.DataFrame({
    'column': actual_dtypes.index,
    'actual_dtype': actual_dtypes.values,
    'check': dtype_check.values
})

print(dtype_df)

              column actual_dtype  check
0               week        int64   True
1       sales_method       object   True
2        customer_id       object   True
3            nb_sold        int64   True
4            revenue      float64   True
5  years_as_customer        int64   True
6     nb_site_visits        int64   True
7              state       object   True


**Data types overview:**

- The data types are as expected when compared to the data validation requirements.

#### 4.1.2 Missing values

In [8]:
# check (already known from step 3.) for missing values
missing_values = data.isnull().sum()
missing_values

week                    0
sales_method            0
customer_id             0
nb_sold                 0
revenue              1074
years_as_customer       0
nb_site_visits          0
state                   0
dtype: int64

**Missing values overview:**
- The check for missing values confirms that there are no missing values except for the 'revenue' column, which has 1074 missing entries

### 4.2 'week'

In [9]:
# check dtype
week_data_type = data['week'].dtype
week_data_type

dtype('int64')

In [10]:
# check missing values
week_missing_values = data['week'].isnull().sum()
week_missing_values

0

In [11]:
# check if the range of week column values is within 1-6 weeks (new product line launched 6 weeks ago)
week_range_valid = data['week'].between(1, 6).all()
week_range_valid

True

**Validation results for 'week' column:**

- The data type of the column 'week' is 'int64', which is appropriate for week numbers
- There are no missing values in this column
- All values are within the valid range of 1 to 6 (six weeks period since new product line launch)

### 4.3 'sales_method'

In [12]:
# check dtype
sales_method_data_type = data['sales_method'].dtype
sales_method_data_type

dtype('O')

In [13]:
# check missing values
sales_method_missing_values = data['sales_method'].isnull().sum()
sales_method_missing_values

0

In [14]:
# check if column values match the expected categories
expected_methods = ['Email', 'Call', 'Email and call']
sales_method_values_valid = data['sales_method'].isin(expected_methods).all()
sales_method_values_valid

False

Not all values in the column match the expected categories ('Email', 'Call' and 'Email and call'). Let's find all distinct entries that are not expected.

In [15]:
# identify unexpected values not included in expected_methods
unexpected_sales_methods = data[~data['sales_method'].isin(expected_methods)]['sales_method'].unique()
unexpected_sales_methods

array(['Email + Call', 'em + call', 'email'], dtype=object)

There are variations and/or typos of the expected categories. We can clean the data by mapping the unexpected sales methods to the correct expected categories.

In [16]:
# correct entries included in the unexpected sales methods to the expected categories
sales_method_corrections = {
    'Email + Call': 'Email and call',
    'em + call': 'Email and call',
    'email': 'Email'
}

# replace unexpected values with the expected category values
data['sales_method'].replace(sales_method_corrections, inplace=True)

# verify correction by checking unique values again
corrected_sales_methods = data['sales_method'].unique()
corrected_sales_methods

array(['Email', 'Email and call', 'Call'], dtype=object)

**Validation results for 'sales_method' columns:**

- The data type is 'object' (denoted as 'O'), which is suitable for categorical data
- There are no missing values in this column
- All variations have now been standardized to the expected categories: 'Email', 'Email and call' and 'Call'

### 4.4 'customer_id'

In [17]:
# check dtype
customer_id_data_type = data['customer_id'].dtype
customer_id_data_type

dtype('O')

In [18]:
# check missing values
customer_id_missing_values = data['customer_id'].isnull().sum()
customer_id_missing_values

0

In [19]:
# check customer_id uniqueness (15000 unique entries)
unique_customer_ids = data['customer_id'].nunique()

# get total number of entries from shape
total_entries = data.shape[0]

# check if the number of unique_customer_ids equals the total number of entries
customer_id_uniqueness = unique_customer_ids == total_entries

unique_customer_ids, customer_id_uniqueness

(15000, True)

We can also check whether all entries are strings of length 36 and if the UUID format is always the same.

In [20]:
# define regular expression pattern for validating UUID format
uuid_pattern = re.compile(r'^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$')

# check if all customer_id entries are strings
all_strings = data['customer_id'].apply(lambda x: isinstance(x, str)).all()

# check if all customer_id entries have length of 36
correct_length = data['customer_id'].apply(lambda x: len(x) == 36).all()

# check if all customer_id entries match the UUID pattern
matches_uuid_pattern = data['customer_id'].str.match(uuid_pattern).all()

all_strings, correct_length, matches_uuid_pattern

(True, True, True)

**Validation results for 'customer_id' column:**

- The data type is 'object' ('O') and is appropriate for unique alphanumeric identifiers with hyphens
- There are no missing values in this column
- All 'customer_id' entries are strings with length of 36 and match the UUID pattern
- Each 'customer_id' is unique and the number of unique ids, 15000, matches the total number of entries in the dataset

### 4.4 'nb_sold'

In [21]:
# check dtype
nb_sold_data_type = data['nb_sold'].dtype
nb_sold_data_type

dtype('int64')

In [22]:
# check missing values
nb_sold_missing_values = data['nb_sold'].isnull().sum()
nb_sold_missing_values

0

In [23]:
# check for non-negative values
nb_sold_non_negative = (data['nb_sold'] >= 0).all()
nb_sold_non_negative

True

**Validation results for 'nb_sold' column:**

- The data type is 'int64', which is correct for integer quantities of sold items
- There are no missing values in this column
- All values are non-negative, which is expected for a sales count
- The values in the column range from 7 (min) to 16 (max), which is plausible (3. Initial basic EDA)

### 4.5 'revenue'

In [24]:
# check dtype
revenue_data_type = data['revenue'].dtype
revenue_data_type

dtype('float64')

In [25]:
# check missing values
revenue_missing_values = data['revenue'].isnull().sum()
revenue_missing_values

1074

In [26]:
# drop NaN values to consider actual numeric entries
non_null_revenues = data['revenue'].dropna()

# check for non-negative values within non-missing values
revenue_non_negative = non_null_revenues.ge(0).all()
revenue_non_negative

True

In [27]:
# check if all non-missing values are rounded to 2 decimal places
revenue_round_check = (non_null_revenues == non_null_revenues.round(2)).all()
revenue_round_check

True

In [28]:
# check min/max values
revenue_min = data['revenue'].min()
revenue_max = data['revenue'].max()

revenue_min, revenue_max

(32.54, 238.32)

**Validation results for 'revenue' column:**

- The data type is 'float64', which is suitable for numerical vlaues with decimals
- There are 1074 missing values (NaN) in this column
- There are no negative values within non-missing values, as expected for revenue
- All non-missing entries are rounded to 2 decimal places as expected
- The values in the column range from 32.54 to 238.32, which is plausible (3. Initial basic EDA)

**Handling missing values:**

- We can decide how to handle the missing values depending on the analysis requirements
- <span style="color:red">Options would include, for example, imputation (i.e. mean/median/mod, regression or KNN), removal or leaving as they are</span>
- Give the lack of clear information about the nature of the missing values and the importance of maintaining data accuracy in fniancial metrics, we'll leave the missing values as they are. This approach allows for an analysis that respects data integrity and provides a clear picture of the dataset's actual state

### 4.6 'years_as_customer'

In [29]:
# check data type
years_as_customer_data_type = data['years_as_customer'].dtype
years_as_customer_data_type

dtype('int64')

In [30]:
# check missing values
years_as_customer_missing = data['years_as_customer'].isnull().sum()
years_as_customer_missing

0

In [31]:
# check for non-negative values
years_as_customer_non_negative = (data['nb_sold'] >= 0).all()
years_as_customer_non_negative

True

In [32]:
# check number of years since founding year
current_year = datetime.now().year
years_since_1984 = current_year - 1984

years_since_1984

39

In [33]:
# check values range for anomalies
years_as_customer_min = data['years_as_customer'].min()
years_as_customer_max = data['years_as_customer'].max()

years_as_customer_min, years_as_customer_max

(0, 63)

The maximum value for this column (63) is greater than the number of years since foundation (39). We need to identify the rows and decide how to approach this anomaly in the data.

In [34]:
# find rows presenting entries greater than maximum allowed
invalid_years_as_customer = data[data['years_as_customer'] > years_since_1984]
invalid_years_as_customer

,week,sales_method,customer_id,nb_sold,revenue,years_as_customer,nb_site_visits,state
13741,2,Email,18919515-a618-430c-9a05-2c7d8fea96af,10,97.22,63,24,California
13800,4,Call,2ea97d34-571d-4e1b-95be-fea1c404649f,10,50.47,47,27,California


There are two entries within the column that need to be addressed. Based on the balance between data integrity and practicality, we need to make sure that values for the column are capped to 39. To preserve the overall distribution and trends within the data, we assume that the most likely cause is a data entry error. The most practical approach, which also minimizes the data distortion when compared to other methods wuch as imputation, is to assign 39 to both entries, which is the cap. This method is easy to understand and justify in this analysis report.

In [35]:
# cap values at 39 years
data['years_as_customer'] = data['years_as_customer'].apply(lambda x: min(x, 39))

# check range again after correction
corrected_years_as_customer_min = data['years_as_customer'].min()
corrected_years_as_customer_max = data['years_as_customer'].max()

corrected_years_as_customer_min, corrected_years_as_customer_max

(0, 39)

**Validation results for 'years_as_customer' column:**

- The data type is 'int64', which is correct and suitable for years
- There are no negative and no missing values in this column
- The range, after addressing the anomalies (capping), is from 0 to 39 years, where the value 0 represents a new customer
- No customer relationship is indicated as longer than the company's existence, as expected

### 4.7 'nb_site_visits'

In [36]:
# check data type
nb_site_visits_data_type = data['nb_site_visits'].dtype
nb_site_visits_data_type

dtype('int64')

In [37]:
# check missing values
nb_site_visits_missing_values = data['nb_site_visits'].isnull().sum()
nb_site_visits_missing_values

0

In [38]:
# check values range and anomalies
nb_site_visits_min = data['nb_site_visits'].min()
nb_site_visits_max = data['nb_site_visits'].max()

nb_site_visits_min, nb_site_visits_max

(12, 41)

**Validation results for 'nb_site_visits' column:**

- The data type is 'int64', which is appropriate for representing counts like website visits
- There are no negative and no missing values in this column
- The range is from 12 to 41 visits, which seems truthful and does not suggest any obvious anomalies or outliers

### 4.8 'state'

In [39]:
# check data type
state_data_type = data['state'].dtype
state_data_type

dtype('O')

In [40]:
# check missing values
state_missing_values = data['state'].isnull().sum()
state_missing_values

0

In [41]:
# check number of unique values
state_number_unique_values = data['state'].nunique()
state_number_unique_values

50

In [42]:
# check unique values
state_unique_values = data['state'].unique()
state_unique_values

array(['Arizona', 'Kansas', 'Wisconsin', 'Indiana', 'Illinois',
       'Mississippi', 'Georgia', 'Oklahoma', 'Massachusetts', 'Missouri',
       'Texas', 'New York', 'Maryland', 'California', 'Tennessee',
       'Pennsylvania', 'North Dakota', 'Florida', 'Michigan',
       'North Carolina', 'Hawaii', 'Colorado', 'Louisiana', 'Virginia',
       'New Mexico', 'Arkansas', 'Alaska', 'Oregon', 'New Hampshire',
       'Ohio', 'New Jersey', 'Connecticut', 'Iowa', 'Montana',
       'Washington', 'Kentucky', 'Alabama', 'Nebraska', 'South Carolina',
       'Minnesota', 'South Dakota', 'Delaware', 'Maine', 'Utah',
       'West Virginia', 'Vermont', 'Rhode Island', 'Nevada', 'Idaho',
       'Wyoming'], dtype=object)

We can check if there have been sales in all 50 US states.

In [43]:
# list of US states
us_states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", 
             "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", 
             "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", 
             "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", 
             "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
             "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", 
             "Wisconsin", "Wyoming"]

# standardize state names for comparison list
data_states_upper = data['state'].str.upper().unique()
us_states_upper = [state.upper() for state in us_states]

# check if all state entries in the column are in the list
consistent_format = all(state in us_states_upper for state in data_states_upper)

# check if there are state entries that are not in the list, if yes display them in a list
states_not_in_list = [state for state in data_states_upper if state not in us_states_upper]

consistent_format, states_not_in_list

(True, [])

**Validation for 'state' column:**

- The data type is 'object' ('O'), which is the correct format for textual data like state names
- There are no missing values in this column
- The format of the entries is consistent
- There are 50 unique values, no indications of misspellings or inconsistencies

## 5. Exploratory data analysis